This notebook calculates the mean & standard deviation MMD for all model in all tasks. It uses pre-generated datasets from each generative model, in which should be placed in a specific place in OS.

In [ ]:
import os
import numpy as np
from collections import defaultdict
from sbibm import get_task
from sbibm.metrics.mmd import mmd
import torch

nb_dir = os.getcwd()

In [ ]:
def calculate_mmd(model:str, task_name:str):    
    task = get_task(task_name)
    reference_samples = task.get_reference_posterior_samples(num_observation=1)

    # Folder containing .npz files
    if model == "dep":
      datasets_path = os.path.join(nb_dir, "cVAE", "Runs - Dependent_cVAE", task_name)
    elif model == "indep":
      datasets_path = os.path.join(nb_dir, "cVAE", "Runs - Independent_cVAE", task_name)
    elif model == "NSF":
      datasets_path = os.path.join(nb_dir, "Flow Based Methods", "Runs - NSF", task_name)
    elif model == "MAF":
      datasets_path = os.path.join(nb_dir, "Flow Based Methods", "Runs - MAF", task_name)
    else:
      print("Model not found")
      return

    # Store MMDs grouped by budget
    mmd_by_budget = defaultdict(list)

    for fname in sorted(os.listdir(datasets_path)):
      file_path = os.path.join(datasets_path, fname)

      # Extract budget from filename (e.g., 'budget_5k')
      try:
          budget = fname.split("budget_")[1].replace(".npz", "")
      except IndexError:
          print(f"Could not extract budget from {fname}, skipping...")
          continue

      # Load generated samples
      data = np.load(file_path)
      if "thetas" in data:
          generated = data["thetas"]
      else:
          print(f"'thetas' key not found in {fname}, skipping...")
          continue

      # Compute MMD and store
      # Convert to torch tensors
      ref_tensor = torch.tensor(reference_samples, dtype=torch.float32)
      gen_tensor = torch.tensor(generated, dtype=torch.float32)

      # Compute MMD
      score = mmd(ref_tensor, gen_tensor)
      
      if score < np.inf:
        mmd_by_budget[budget].append(score)

    # Print mean and std MMDs per budget
    print(f"\nTask: {task_name}\nMMD Summary (Mean ± Std):")
    for budget in sorted(mmd_by_budget.keys(), key=lambda b: int(b.replace("k", ""))):
        scores = np.array(mmd_by_budget[budget])
        mean_score = scores.mean()
        std_score = scores.std()
        print(f"Budget {budget}: MMD = {mean_score:.3f} ± {std_score:.3f} [Nr. Scores used: {len(scores)}]")

In [ ]:
tasks = [
"gaussian_linear",
"gaussian_linear_uniform",
"slcp",
"slcp_distractors",
"bernoulli_glm",
"bernoulli_glm_raw",
"gaussian_mixture",
"two_moons",
"sir",
"lotka_volterra"
 ]

# Dependent cVAE

In [ ]:
for task_name in tasks:
    calculate_mmd("dep", task_name)

# Independent cVAE

In [ ]:
for task_name in tasks:
    calculate_mmd("indep", task_name)

# Masked Autoregressive Flows (MAF)

In [ ]:
for task_name in tasks:
    calculate_mmd("MAF", task_name)

# Neural Spline Flows

In [ ]:
for task_name in tasks:
    calculate_mmd("NSF", task_name)